In [54]:
import pandas as pd

# Analiza receptov v računalniški igri Minecraft

### Uvod in razlage pojmov

Ta projekt iz glavne wiki spletne strani za računalniško igro Minecraft postrga vse recepte za izdelavo predmetov. Te nato shrani v CSV tabelo in jih analizira, a o tem kasneje.

Najprej se dogovorimo za nekaj izrazov:
- Material - predmet, ki je uporabljen v izdelavi drugega predmeta
- Predmet - predmet, ki ga poskušamo sestaviti iz materialov
- Recept - razporeditev do devetih matrialov v kvadratu velikosti 3x3, iz česar nato lahko dobimo nov predmet. Nam ni pomembna sama razporeditev, le koliko je potrebnega vsakega materiala

Prvi korak je naložiti HTML spletne strani na računalnik in ga predelati v datoteko CSV. To naredimo tako, da zaženemo program `main.py`, a sem na repozitorij objavil vse vmesne datoteke, saj je del programa, ki naloži HTML nekoliko nezanesljiv (zelo odvisen od internetne povezave, verzije iskalnika Chrome ipd.). Če vam pri zagonu `main.py` selenium vrača napako, nastavite spremenljivko na začetku programa na `download_from_web = False`.

In [55]:
csv_data = pd.read_csv("data/data.csv").sort_values("Name").sort_values("Section")
csv_data

,Section,Name,Ingredients,Amounts,Yield
344,Brewing,Fermented_Spider_Eye,Brown_Mushroom;Spider_Eye;Sugar,1;1;1,1
343,Brewing,Cauldron,Iron_Ingot,7,1
342,Brewing,Brewing_Stand,Blaze_Rod;Cobblestone,1;2,1
345,Brewing,Glass_Bottle,Glass,3,3
341,Brewing,Blaze_Powder,Blaze_Rod,1,2
...,...,...,...,...,...
308,Utilities,Clock,Gold_Ingot;Redstone_Dust,4;1,1
309,Utilities,Compass,Iron_Ingot;Redstone_Dust,4;1,1
310,Utilities,Lead,Slimeball;String,1;4,2
311,Utilities,Recovery_Compass,Compass;Echo_Shard,1;8,1


Poglejmo si tudi, kateri recepti proizvedejo največ predmetov oz. imajo največji donos:

In [56]:
csv_data.sort_values("Yield", ascending=False)

,Section,Name,Ingredients,Amounts,Yield
195,Decoration_blocks,Iron_Bars,Iron_Ingot,6,16
229,Decoration_blocks,Stained_Glass_Pane,White_Stained_Glass,6,16
279,Transportation,Rail,Iron_Ingot;Stick,6;1,16
188,Decoration_blocks,Glass_Pane,Glass,6,16
358,Materials,Gold_Ingot,Block_of_Gold,1,9
...,...,...,...,...,...
161,Decoration_blocks,Banner,Stick;White_Wool,1;6,1
231,Decoration_blocks,Stonecutter,Iron_Ingot;Stone,1;3,1
162,Decoration_blocks,Barrel,Oak_Planks;Oak_Slab,5;2,1
173,Decoration_blocks,Chest,Oak_Planks,8,1


Vidimo, da imajo železne rešetke, tračnice in vitraži ter prozorne steklene plošče vse najvišji donos, kar je 16 iz enega recepta.

### Primarni in sekundarni materiali

Sedaj bi radi definirali razliko med materiali: tisti, ki jih lahko naredimo iz česa drugega in tisti, ki jih ne moremo (na primer drva, kamen ipd.). Za to bomo definirali razred, ki bo hranil podatke o predmetih, da jih lahko nato primerjamo:

In [57]:
class Item():
    def __init__(self, section: str, name: str) -> None:
        self.name = name
        self.section = section
        self.recipes = []
        self.material_for = set()
        self.is_primary = False
        self.is_final = False
        self.max_depth = 0
        self.primary_ingredients = []
        # For each primary material, there is a tuple in the form of (Ingredient, Amount)
        self.items_crafted = []
        # A list of all the items, that can be crafted from this (directly or indirectly)
        self.recipe_chains = []
        # Contains lists of items, ranging from this item to its primary materials
        self.max_depth = 0
    
    def new_recipe(self, ings: str, ams: str, y: str):
        recipe = {}
        ings = ings.split(";")
        ams = ams.split(";")
        for i in range(len(ams)):
            recipe[ings[i]] = ams[i]
        recipe["yield"] = y
        self.recipes.append(recipe)
    
    def new_material(self, item: str):
        self.material_for.add(item)
    
    def __str__(self) -> str:
        out = self.name + " is "
        if self.is_final:
            out += "final" 
        elif self.is_primary:
            out += "primary"
        else:
            out +="intermediate"
        out += "\nItems crafted from this are: "
        for item in self.items_crafted:
            out += f"{item}, "
        out = out[:-2] + "\nThe necessary primary materials for this item are: (material, amount per produced item)\n"
        for material in self.primary_ingredients:
            out += f"({material[0]}, {str(round(material[1], 3))})\n"
        out += f"The longest chain of crafting operations for this item is {self.max_depth} operations long.\n"
        out += "#" * 50
        return out


Sedaj shranimo vsak predmet v CSV tabeli v svoj predmet razreda Item in te shranimo v slovar, da lahko lažje dostopamo do njih:

In [58]:
items = {}

for i, row in csv_data.iterrows():
    #print(row)
    items[row["Name"]] = Item(row["Section"], row["Name"])
    items[row["Name"]].new_recipe(row["Ingredients"], row["Amounts"], row["Yield"])

Vsak predmet razreda Item hrani podatke o specifičnem predmetu iz CSV datoteke in potem še dodaten podatek: za katere predmete je uporabljen.

Pri iskanju primarnih materialov je še ena ovira; nekatere primarne materiale je mogoče "stisniti" v kocko tega materiala, to kocko pa je potem mogoče razdreti nazaj na prvotno količino materiala (Na primer železo: 9 palic železa lahko spremenimo v kocko železa, to pa lahko spremenimo nazaj v 9 železnih palic).

Takšnih materialov je več, tako da jih poiščimo. Za vsak predmet bomo iskali, če je ta predmet material za katerega od svojih materialov:

In [59]:
for item in items.values():
    for recipe in item.recipes:
        for ingredient in recipe.keys():
            if ingredient not in items.keys():
                continue
            for recipe2 in items[ingredient].recipes:
                if item.name in recipe2.keys():
                    print(item.name)

Hay_Bale
Bone_Block
Block_of_Gold
Block_of_Iron
Slime_Block
Honey_Block
Honey_Bottle
Wheat
Bone_Meal
Iron_Ingot
Gold_Ingot
Slimeball
Block_of_Redstone
Redstone_Dust


Sedaj izmed teh izberemo tiste, ki jih nastavimo kot primarne:

In [60]:
primaries = ["Dried_Kelp", "Raw_Iron", "Raw_Gold", "Wheat", "Raw_Copper",
             "Coal", "Diamond", "Iron_Ingot", "Gold_Ingot", "Emerald", "Bone_Meal",
             "Lapis_Lazuli", "Netherite_Ingot", "Slimeball", "Honey_Block", "Redstone_Dust"]

Sedaj, ko imamo vse predmete shranjene, lahko za vsak predmet preverimo, v katerem receptu je material za drug predmet:

In [61]:
materials = {}

for item in items.keys():
    for recipe in items[item].recipes:
        for material in recipe.keys():
            if material == "yield":
                continue
            if material not in items.keys():
                # Če material v receptu še ni v slovarju predmetov, to pomeni,
                # da nima recepta, ki bi naredil ta predmet, torej je primarni material
                materials[material] = Item("Primary", material)
                materials[material].is_primary = True
                if material not in primaries:
                    primaries.append(material)
                materials[material].new_material(item)
            elif material in primaries:
                items[material].is_primary = True
            # Vsakem material v vsakem receptu zabeležimo, da je iz njega mogoče narediti predmet, ki je posledica tega recepta
            else:
                items[material].new_material(item)

items = materials | items

{'Fermented_Spider_Eye': <__main__.Item object at 0x00000205B8E96A90>, 'Cauldron': <__main__.Item object at 0x00000205B8ECF130>, 'Brewing_Stand': <__main__.Item object at 0x00000205B8E634C0>, 'Glass_Bottle': <__main__.Item object at 0x00000205B8E96910>, 'Blaze_Powder': <__main__.Item object at 0x00000205B8E637F0>, 'Magma_Cream': <__main__.Item object at 0x00000205B8E96AC0>, 'Glistering_Melon_Slice': <__main__.Item object at 0x00000205B8E968E0>, 'Polished_Diorite_Slab': <__main__.Item object at 0x00000205B8E96640>, 'Deepslate_Tiles': <__main__.Item object at 0x00000205B8E96040>, 'Stone_Brick_Stairs': <__main__.Item object at 0x00000205B8E63CA0>, 'Diorite': <__main__.Item object at 0x00000205B8E63640>, 'Diorite_Slab': <__main__.Item object at 0x00000205B8E63130>, 'Diorite_Stairs': <__main__.Item object at 0x00000205B8E63E50>, 'Stone_Brick_Slab': <__main__.Item object at 0x00000205B8E632E0>, 'Dripstone_Block': <__main__.Item object at 0x00000205B8C78100>, 'Nether_Brick_Stairs': <__main__.

Preverimo, da program pravilno zazna vse primarne materiale (če predmet nima recepta in je v receptu, potem je primarni material):

In [62]:
print("Predmeti, ki nimajo recepta in niso primarni:")
for item in items.keys():
    if len(items[item].recipes) == 0 and not items[item].is_primary:
        print(item, items[item].material_for)
print("Konec!")

Predmeti, ki nimajo recepta in niso primarni:
Konec!


Ker je presek med množico predmetov, ki niso primarni materiali, in množico predmetov brez recepta, prazna, naš program deluje pravilno.

Sedaj lahko zelo preprosto tudi definiramo končne predmete, to so tisti predmeti, ki niso material za noben drug predmet:

In [63]:
for item in items.values():
    if len(item.material_for) == 0:
        item.is_final = True

Tega ne bomo preverjali, saj precej očitno deluje.

## Veriga receptov

Sedaj, ko imamo te podatke na voljo, lahko tvorimo verige receptov. Zanima nas, Katere vse materiale moramo narediti, da izdelamo vsak predmet. Poleg tega bi bilo priročno vedeti, koliko katerih primarnih materialov potrebujemo za poljuben predmet, in za katere predmete moramo uporabiti posamezen primarni element.

In [64]:
def recursive_recipe_search(name: str, multiplier: float=1) -> list:
    if items[name].is_primary:
        return []
    out = []
    for i, recipe in enumerate(items[name].recipes):
        out.append([])
        if len(recipe.keys()) == 0:
            continue
        for material in recipe.keys():
            if material == None:
                return []
            if material == "yield":
                continue
            mult = multiplier * int(recipe[material]) / int(recipe["yield"])
            if mult == None:
                return []
            out[i].append((material, mult))
            out[i] += recursive_recipe_search(material, mult)
    return out

for item in items.values():
    if not item.is_primary:
        item.recipe_chains = recursive_recipe_search(item.name)

Sedaj imamo za vsak predmet verigo materialov v obliki gnezdenih seznamov. Iz tega lahko preberemo, katere primarne materiale potrebujemo (in koliko vsakega) za vsak predmet in koliko korakov potrebujemo, da pridemo od primarnih materialov do želenega predmeta:

In [65]:
def find_primaries_and_depth(recipe_chain: list, depth: int=0):
    out = []
    for link in recipe_chain:
        if type(link) == type([]):
            temp = find_primaries_and_depth(link)
            out += temp[0]
            depth = temp[1]
        else:
            if items[link[0]].is_primary:
                out.append(link)
    
    return out, depth + 1

for item in items.values():
    out = []
    for chain in item.recipe_chains:
        temp = find_primaries_and_depth(chain)
        out += temp[0]
        item.max_depth = max(item.max_depth, temp[1])
    for i, tup in enumerate(out):
        for j in range(i):
            if out[j][0] == tup[0]:
                if out[j][1] > tup[1]:
                    out.pop(i)
                else:
                    out.pop(j)
    for mat, amt in out:
        amt = float.as_integer_ratio(amt)
    item.primary_ingredients = out

Nazadnje poglejmo še, katere predmete lahko (posredno ali neposredno) naredimo iz vsakega predmeta:

In [66]:
def recursive_material_search(item: str):
    out = set()
    for material in items[item].material_for:
        out.add(material)
        if not items[material].is_final:
            out = out.union(recursive_material_search(material))
    return out

for item in items.keys():
    items[item].items_crafted = recursive_material_search(item)

Sedaj dodamo razredu Item funkcijo `__str__`, da lahko uporabimo `print(predmet)`, da vidimo vse nabrane podatke o predmetu:
(To bom onemogočil, saj je sporočilo, ki ga izpiše, zelo dolgo. Če ga vseeno želite videti, spremenite `False` v `True` in za\enite celico)

In [67]:
if False:
    for item in items.values():
        print(item)

In [68]:
for item in items.values():
    if item.primary_ingredients == []:
        item.is_primary = True
    elif item.items_crafted == set():
        item.is_final = True

Vse te podatke shranimo še v novo CSV datoteko:

In [69]:

out = "Item,Position,Primaries,Primary_Amounts,Crafts,Chain\n"
for item in items.values():
    out += item.name + ","
    out += "Primary" if item.is_primary else ("Final" if item.is_final else "Intermediate")
    out += ","
    if item.is_primary:
        out += "_,_,"
        for craft in item.items_crafted:
            out += craft
            out += ";"
        out = out[:-1] + ",0"
    elif item.is_final:
        for primary in item.primary_ingredients:
            out += primary[0] + ";"
        out = out[:-1] + ","
        for primary in item.primary_ingredients:
            amt_rounded = round(primary[1], 3)
            if amt_rounded % 1 == 0:
                amt_rounded = int(amt_rounded)
            out += str(amt_rounded) + ";"
        out = out[:-1] + ",_," + str(item.max_depth)
    else:
        for craft in item.items_crafted:
            out += craft
            out += ";"
        out = out[:-1]
        out += "," + str(item.max_depth)
        print(item.max_depth)
    out += "\n"
out.strip("\n")
with open("data/extracted.csv", "w", encoding="utf8") as file:
    file.write(out)
    

1
1
2
3
1
2
2
2
1
1
2
2
1
1
2
2
2
1
2
1
1
1
2
2
1
1
2
1
1
1
1
3
1
1
1
1
1
1
1
2
3
1
2
3
1
1
3
2
1
2
1
1
1
2
2
2
1
2
1
2
1
1
3
1
1
1
1
1
1
1
1
1
2
2
1
1
2
1
2
1
2
2
1
1
1


Nazadnje pa si poglejmo še nekaj zanimivih grafov:

In [70]:
csv_extr = pd.read_csv("data/extracted.csv").sort_values("Item").sort_values("Position", ascending=False)
csv_extr

,Item,Position,Primaries,Primary_Amounts,Crafts,Chain
390,Iron_Ingot,Primary,_,_,0,NaN
92,Rabbit_Hide,Primary,_,_,Leather_Boots;Leather_Horse_Armor;Enchanting_T...,0.0
85,Cooked_Rabbit,Primary,_,_,Rabbit_Stew,0.0
63,Glowstone_Dust,Primary,_,_,Spectral_Arrow,0.0
400,Gold_Ingot,Primary,_,_,0,NaN
...,...,...,...,...,...,...
284,Golden_Chestplate,Final,Gold_Ingot,8,_,1.0
371,Golden_Carrot,Final,Carrot;Gold_Ingot,1;0.889,_,2.0
283,Golden_Boots,Final,Gold_Ingot,4,_,1.0
369,Golden_Apple,Final,Apple;Gold_Ingot,1;8,_,1.0
